In [1]:
import torch
import cv2
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry, SamPredictor
import numpy as np
import matplotlib.pyplot as plt
import glob
import torch
import clip
from PIL import Image
from tqdm import tqdm
import functions as fnc
from importlib import reload
import gc
from skimage.measure import label, regionprops

In [3]:
OutDIR='/DATA/vito/output/'
DataDIR='/DATA/vito/data/'
#fn_img = glob.glob(DataDIR+'test_img/*')
fn_img = glob.glob(DataDIR+'drone_ortho/*')
fn_img.sort()
print('Available images:')
for i, fn in enumerate(fn_img):
    print(f'{i}:' + fn.split("/")[-1])
print("---------------")

Available images:
0:512patched_LaPaya.npy
1:512patched_mask.npy
2:LaPaya_Apr2024_03May24_orthomosaic_5cm_UTM19S_WGS84.tif
3:LaPaya_Apr2024_orthomosaic_5cm_UTM19S_WGS84.tif
4:Ravi1_DEM_0.6mm.tif
5:Ravi1_Orthomosaic_0.2mm.tif
6:Ravi1_orthomosaic_from_DEM_0.6mm.tif
7:Ravi2_Orthomosaic_0.2mm.tif
8:Ravi2_Orthomosaic_0.2mm_clip1.tif
9:Ravi3_Orthomosaic_0.2mm.tif
10:big_clip.tif
11:small_clip1.tif
12:small_clip2.tif
---------------


In [4]:
img_select=int(input("Select a image: "))
image = cv2.imread(fn_img[img_select])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(fn_img[img_select].split("/")[-1]+' imported')
print('Image size:', image.shape)

Ravi2_Orthomosaic_0.2mm_clip1.tif imported
Image size: (2680, 5125, 3)


In [ ]:
#load mask

In [ ]:
#set_overlap = set()

# Iterate through each column of the array
#for i in range(ar_masks_flat.shape[1]):
    # for each pixel find out the idex of mask where the pixel was in a mask
#    nz = np.where(ar_masks_flat[:, i] != 0)[0]
    
    # if there are overlap of mask
#    if len(nz) > 1:
#        nz_tuple = tuple(nz)
#        set_overlap.add(nz_tuple)

# Convert the set back to a list if needed
#list_overlap = list(set_overlap)

#find pixel wise overlaps
#list_overlap=[]
#for i in range(ar_masks_flat.shape[1]):
#    nz=np.nonzero(ar_masks_flat[:,i])[0]#for each pixel find out the idex of mask where the pixel was in a mask
#    if len(nz)>1:#if there are overlap of mask
#        if not any(np.array_equal(nz, arr) for arr in list_overlap):
#            list_overlap.append(nz)
#list_overlap1=[ np.nonzero(ar_masks_flat[:,i])[0] for i in range(ar_masks_flat.shape[1]) if len(np.nonzero(ar_masks_flat[:,i])[0])>1]

In [ ]:
list_of_masks = [fnc.clean_mask(mask['segmentation'].astype(np.uint8)) for mask in masks]#remove small disconnected parts
ar_masks=np.stack(list_of_masks)
ar_masks_flat=ar_masks.reshape((ar_masks.shape[0],ar_masks.shape[1]*ar_masks.shape[2]))#flat 2d to 1d masks

#find pixel wise overlaps
list_overlap=[]
# Iterate through each column of the array
for i in range(ar_masks_flat.shape[1]):
    # for each pixel find out the idex of mask where the pixel was in a mask
    nz = np.where(ar_masks_flat[:, i] != 0)[0]
    
    # if there are overlap of mask
    if len(nz) > 1:
        list_overlap.append(tuple(nz))

#get uniqe pairs and intersection area(pixel) for each pair
#list_overlap_intersection=[np.unique(np.sum([list_of_masks[i] for i in overlap],axis=0), return_counts=True)[1][-1] for overlap in list_overlap]
group_counter=Counter(list_overlap)
unique_groups = [list(tup) for tup in group_counter.keys()]
group_overlap_area = list(group_counter.values())

#filter by intersection area
threshold=1000
filtered=np.array(group_overlap_area)>threshold
unique_groups_thresholded=[unique_groups[i] for i in range(len(unique_groups)) if filtered[i]]
#list_overlap_threshold=[list_overlap[i] for i in range(len(list_overlap_intersection)) if list_overlap_intersection[i] > threshold]

#report filter
print(f'Threshold: {threshold} pixels, {len(list_overlap)-len(unique_groups_thresholded)} groups removed',
      '\nOverlap groups before filtering: {len(list_overlap)}, after filtering: {len(unique_groups_thresholded)}')

In [ ]:
#groupping overlaps

#check if there is remaining ungroupped pairs
checker=np.zeros(len(masks))
for gp in list_overlap_threshold:
    for i in gp:
        checker[i]+=1
remaining_ungroupped=np.max(np.unique(checker))

cleaned_groups=None#dummy to identify first run

while remaining_ungroupped>1:#keep groupping until there is no overlapping groups
    groups=[]

    if not cleaned_groups:#the first run
        temp_list_overlap=list_overlap_threshold.copy()
    else:#>2 run
        temp_list_overlap=cleaned_groups.copy()

    while len(temp_list_overlap)>1:
        temp_group=temp_list_overlap[0].copy()
        remaining=[]
        for i in np.arange(1,len(temp_list_overlap),1):
            num_common_val=len(np.intersect1d(temp_list_overlap[0],temp_list_overlap[i]))
            if num_common_val!=0:
                temp_group=np.hstack((temp_group,temp_list_overlap[i]))
            else:
                remaining.append(temp_list_overlap[i])
        temp_list_overlap=remaining.copy()
        groups.append(temp_group)
    groups.append(remaining[0])

    #remove duplicated index after stacking
    cleaned_groups=[np.unique(group) for group in groups]

    checker=np.zeros(len(masks))
    for gp in cleaned_groups:
        for i in gp:
            checker[i]+=1
    remaining_ungroupped=np.max(np.unique(checker))

In [ ]:
print(f'{len(cleaned_groups)} groups found\n')
if len(np.unique(np.hstack(cleaned_groups)))==len(np.unique(np.hstack(list_overlap))):
    if np.sum(np.unique(np.hstack(cleaned_groups))==np.unique(np.hstack(list_overlap))):
        print('nothing missing')
    else:
        print('Warning: something missing')
else:
    print('Warning: total number not match')

In [ ]:
for i in cleaned_groups[0]:
    print(i)
    print((regionprops(label(list_of_masks[i])))[0].area)

In [ ]:
mask1=list_of_masks[62]
mask2=list_of_masks[51]
intersection = np.logical_and(mask1, mask2)

# Calculate the union
union = np.logical_or(mask1, mask2)

# Calculate the IoU
iou = np.sum(intersection) / np.sum(union)
iou

In [ ]:
#plot group
import matplotlib.gridspec as gridspec
k=3
# Create a figure
fig = plt.figure(figsize=(20, 20))

num_rows=int(np.ceil((len(cleaned_groups[k])/3)))+1
gs = gridspec.GridSpec(num_rows, 3, height_ratios=list((np.ones(num_rows).astype(int))))

ax_big = fig.add_subplot(gs[0, :])
ax_big.imshow(shrinked_image)
ax_big.imshow(np.sum(np.stack([list_of_masks[i] for i in cleaned_groups[k]]),axis=0), alpha=0.3)
ax_big.set_title('RGB overlay')

j=0
for i in range(len(cleaned_groups[k])):
    ax = fig.add_subplot(gs[(i//3)+1, j])
    ax.imshow(list_of_masks[cleaned_groups[k][i]])
    ax.set_title(f'{cleaned_groups[k][i]}')

    if j==2:
        j=0
    else:
        j+=1

plt.tight_layout()
plt.show()

In [ ]:
mask1=list_of_masks[cleaned_groups[k][1]]
mask2=list_of_masks[cleaned_groups[k][6]]
intersection = np.logical_and(mask1, mask2)

np.sum(intersection)
#base on l band check if these are bright side and shadow side and by comparing a b band see if they are identical?

In [ ]:
target=60
overlaps_with_target=[]
for ls in list_overlap:
    if np.any(target == ls[0]):
        overlaps_with_target.append(ls[0])

In [ ]:
overlaps_with_target=np.unique(np.hstack(overlaps_with_target))
overlaps_with_target=overlaps_with_target[overlaps_with_target != target]
overlaps_with_target

In [ ]:
np.unique(label(ar_masks[60]))

In [ ]:
labels = label(ar_masks[60])
regions = regionprops(labels)

# Sort regions by area
sorted_regions = sorted(regions, key=lambda x: x.area, reverse=True)

# Print sorted regions by area
for region in sorted_regions:
    print(f"Label: {region.label}, Area: {region.area}")

In [ ]:
plt.imshow(label(ar_masks[60])==2)

In [ ]:
plt.imshow(np.sum(ar_masks[overlaps_with_target],axis=0))